<a href="https://colab.research.google.com/github/iannashon/cse6250/blob/main/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 68.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2c71abe9d6e800ad7c8238a2bb6d79e9f688676a077ca325421748b5dc5e0a57
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pyspark as pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SQLContext
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import array, lit
from pyspark.sql import Window, functions as F

##Setup

In [ ]:
spark = SparkSession\
    .builder\
    .master('local')\
    .appName('cse6250Project')\
    .enableHiveSupport() \
    .getOrCreate()

##1. Identifying Heart Failure Admissions 

In [ ]:
# -- load admissions and diagnosis_icd tables
admissions = spark.read.option("header",True).csv("drive/MyDrive/mimicdb/data/ADMISSIONS.csv")
diagnosis_icd = spark.read.option("header",True).csv("drive/MyDrive/mimicdb/data/DIAGNOSES_ICD.csv")

admissions.show(1, vertical=True)
diagnosis_icd.show(1, vertical=True)

# -- Merge admissions & diagnosis_icd tables. select columns and filter admissions to identify only those associated with relevant diagnoses
diagnosis_icd = diagnosis_icd.withColumnRenamed("HADM_ID", "HADM_ID_X")
admission_icd9 = diagnosis_icd.join(admissions, diagnosis_icd.HADM_ID_X == admissions.HADM_ID, "left").select(admissions["*"], diagnosis_icd["ICD9_CODE"])
admission_icd9 = admission_icd9.select("HADM_ID", "SUBJECT_ID", "ICD9_CODE",  "ADMITTIME", "DISCHTIME", "DIAGNOSIS" )
# admission_icd9.count()

# Filter heart failure ICD-9 codes
list_of_icd_codes = ["39891", "40201", "40211", "40291","40401", "40403", "40411", "40413", "40491", "40493", "4280", "4281",
"42820", "42821", "42822", "42823", "42830", "42831", "42832", "42833","42840", "42841", "42842", "42843",  "4289"]
admission_for_hf = admission_icd9.where(admission_icd9.ICD9_CODE.isin(list_of_icd_codes))
admission_for_hf = admission_for_hf.dropDuplicates(["HADM_ID"])
w_spec  = Window.partitionBy("SUBJECT_ID").orderBy("ADMITTIME")
admission_x = admission_for_hf.withColumn("row_number", row_number().over(w_spec)).sort(desc("SUBJECT_ID"), "row_number")
admission_lead = admission_x.withColumn("lead", -1 * datediff(admission_x.DISCHTIME, lead("ADMITTIME", 1).over(w_spec)))

# Add isReadmission
admission_status = admission_lead.withColumn("isReadmission", 
                                           when(col("lead").isNull(), False)
                                           .otherwise(True)).sort(desc("SUBJECT_ID"), asc("row_number"))

# Add isEarlyReadmission
admission_status = admission_status.withColumn("isEarlyReadmission", 
                                           when(col("lead")<=30, True)
                                           .otherwise(False)).sort(desc("SUBJECT_ID"), asc("row_number"))                                          


-RECORD 0------------------------------------
 ROW_ID               | 21                   
 SUBJECT_ID           | 22                   
 HADM_ID              | 165315               
 ADMITTIME            | 2196-04-09 12:26:00  
 DISCHTIME            | 2196-04-10 15:54:00  
 DEATHTIME            | null                 
 ADMISSION_TYPE       | EMERGENCY            
 ADMISSION_LOCATION   | EMERGENCY ROOM ADMIT 
 DISCHARGE_LOCATION   | DISC-TRAN CANCER/... 
 INSURANCE            | Private              
 LANGUAGE             | null                 
 RELIGION             | UNOBTAINABLE         
 MARITAL_STATUS       | MARRIED              
 ETHNICITY            | WHITE                
 EDREGTIME            | 2196-04-09 10:06:00  
 EDOUTTIME            | 2196-04-09 13:24:00  
 DIAGNOSIS            | BENZODIAZEPINE OV... 
 HOSPITAL_EXPIRE_FLAG | 0                    
 HAS_CHARTEVENTS_DATA | 1                    
only showing top 1 row

-RECORD 0------------
 ROW_ID     | 1297   
 SUBJECT_ID 

In [ ]:
admission_status.count()

14040

In [ ]:
#-- OJAS cleaning check
# admission_for_hf.show(5)
# admission_lead.filter(col("lead")==0).orderBy("SUBJECT_ID","HADM_ID").show(10)
# admission_status.show(5)

NameError: ignored

In [ ]:
noteevents = spark.read.option("multiLine",True).option("header",True).csv("drive/MyDrive/mimicdb/data/NOTEEVENTS.csv")
# noteevents.show(1, vertical=True)

# Calculate Text length
note_events_wds= noteevents.filter(noteevents.CATEGORY == 'Discharge summary')
note_events_wds= note_events_wds.withColumn("TEXT_LN_OLD", length("TEXT"))
note_events_wds= note_events_wds.withColumn("TEXT_LN", F.size(F.split(note_events_wds['TEXT'], ' ')))
note_events = note_events_wds.dropDuplicates()


w_espec  = Window.partitionBy("HADM_ID").orderBy("TEXT_LN")
note_events_wds = note_events_wds.withColumn("max_txt_lenght", row_number().over(w_espec)).sort(desc("TEXT_LN"))
# note_events_wds.orderBy(desc("HADM_ID")).show(5)

windowDept = Window.partitionBy("HADM_ID").orderBy(col("TEXT_LN").desc())
note_events_cln = note_events_wds.withColumn("row",row_number().over(windowDept)) \
  .filter(col("row") == 1).drop("row") 

admissions = admission_status.join(note_events_cln, admission_status.HADM_ID == note_events_cln.HADM_ID, "left").select(admission_status["*"], note_events_cln["CATEGORY"] , note_events_cln["TEXT"] )
admissions= admissions.withColumn("TEXT_LN_OLD", length("TEXT"))
admissions= admissions.withColumn("TEXT_LN", F.size(F.split(note_events_wds['TEXT'], ' ')))
admissions.count()


14040

In [ ]:
#-- OJAS cleaning check
note_events_wds.show(3)
note_events_cln.show(3)
admissions.show(3)

+------+----------+-------+----------+---------+---------+-----------------+-----------+----+-------+--------------------+-------+-------------+--------------+
|ROW_ID|SUBJECT_ID|HADM_ID| CHARTDATE|CHARTTIME|STORETIME|         CATEGORY|DESCRIPTION|CGID|ISERROR|                TEXT|TEXT_LN|TEXT_LN_WORDS|max_txt_lenght|
+------+----------+-------+----------+---------+---------+-----------------+-----------+----+-------+--------------------+-------+-------------+--------------+
|  1484|     66807| 166588|2199-05-23|     null|     null|Discharge summary|     Report|null|   null|Admission Date:  ...|  50663|         7064|             1|
| 34099|     28941| 107962|2144-09-18|     null|     null|Discharge summary|     Report|null|   null|Admission Date:  ...|  50035|         7979|             1|
| 48774|     51821| 197028|2161-01-01|     null|     null|Discharge summary|     Report|null|   null|Admission Date:  ...|  49417|         6534|             1|
+------+----------+-------+----------+--

In [ ]:
admissions.filter(admissions.isEarlyReadmission=='true').count()

958

In [ ]:
admissions.toPandas().to_csv("drive/MyDrive/mimicdb/dataset_05.csv", header=True)